In [ ]:
# Import libraries
import pandas as pd

In [ ]:
# Get prices dataset (y)
merged_df = pd.read_csv("./output/return.csv",index_col=[0, 1]).sort_index()
print(f"Stock grwoth dataset shape: {merged_df.shape}")

# Get fundamental factors
fund_ful = pd.read_csv("./output/fundamentals.csv",index_col=[0, 1]).sort_index()
merged_df = pd.merge(merged_df, fund_ful, left_index=True, right_index=True, how='inner')
print(f"Technical factor dataset shape: {fund_ful.shape}")

# Get fundamental growth rates
fund_gro = pd.read_csv("./output/fundamentals_growth.csv",index_col=[0, 1]).sort_index()
fund_gro = fund_gro.add_suffix(" (Growth)")
merged_df = pd.merge(merged_df, fund_gro, left_index=True, right_index=True, how='inner')
print(f"Fundamental factor dataset shape: {fund_gro.shape}")

# Get techncical factors
techn = pd.read_csv("./output/technical.csv",index_col=[0, 1]).sort_index()
merged_df = pd.concat([merged_df, techn], axis=1, ignore_index=False)
print(f"Technical factor dataset shape: {techn.shape}")

In [ ]:
# Process and remove NaN values from the dataset
processing = 3

# Fill all the NaNs: Full Fill
if (processing == 1):
    clean_df = merged_df.ffill()
    clean_df = clean_df.bfill()
    clean_df = clean_df.dropna()
    
# Do half-and-half: Partial Fill
elif (processing == 2):
    clean_df = merged_df.dropna(thresh=merged_df.shape[0]*0.7, axis=1)
    clean_df = clean_df.ffill(limit=5)
    clean_df = clean_df.bfill(limit=5)
    clean_df = clean_df.dropna()

# Delete all the NaNs: No Fill
elif (processing == 3):
    clean_df = merged_df.dropna(thresh=merged_df.shape[0]*0.7, axis=1)
    clean_df = clean_df.dropna()

# Something bad has happened
else:
    print("Error!")

# Preview the cleaned dataset so far
clean_df.head()

In [ ]:
# Turn quarterly times into datetimes
times = [(pd.to_datetime(x[:4]) + pd.offsets.QuarterBegin(int(x[5:]))).strftime('%m/%d/%Y') for x in clean_df.index.get_level_values(1)]
clean_df.index = clean_df.index.set_levels(times, level=1, verify_integrity=False)

clean_df.columns = [col.replace(',', '') for col in clean_df.columns]
clean_df.columns = [col.replace('"', '') for col in clean_df.columns]

In [ ]:
# Turn return into itself * 100
clean_df['Return'] = clean_df['Return'].apply(lambda x: x*100)

# Sort by date
clean_df = clean_df.sort_index(level=1)

In [ ]:
# Print the merged dataset
print(f"Final dataset shape: {clean_df.shape}")
clean_df.head()

In [ ]:
# Save the merged dataset
clean_df.to_csv('./output/complete_dataset.csv', float_format='%.2f')